<a href="https://colab.research.google.com/github/darthgera123/Object-Detection/blob/master/EffDet_MASKD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MASKD EfficientDet Training On COCO Dataset
This dataset and notebook correspond to the [MASKD Challenge](https://www.aicrowd.com/challenges/aicrowd-blitz-2/problems/maskd) being held on [AIcrowd](https://www.aicrowd.com/).

EfficientDet is one of the SOTA in Object detection with Efficient Net as its backbone.We use this [implementation](https://github.com/zylo117/Yet-Another-EfficientDet-Pytorch) for our purposes. Read more about the network [here](https://towardsdatascience.com/efficientdet-scalable-and-efficient-object-detection-review-4472ffc34fd9) 

## Install Libraries



In [ ]:
!pip install pycocotools numpy==1.16.0 opencv-python tqdm tensorboard tensorboardX pyyaml webcolors matplotlib
!pip install torch==1.4.0
!pip install torchvision==0.5.0

In [ ]:
import os
import sys
if "projects" not in os.getcwd():
  !git clone --depth 1 https://github.com/zylo117/Yet-Another-EfficientDet-Pytorch
  os.chdir('Yet-Another-EfficientDet-Pytorch')
  sys.path.append('.')
else:
  !git pull


# download pretrained weights
! wget https://github.com/zylo117/Yet-Another-EfficientDet-Pytorch/releases/download/1.0/efficientdet-d0.pth -O weights/efficientdet-d0.pth


## Download and Prepare Dataset
The dataset is supposed to be in a given format. Also we need to create a `projects/masks.yml` which gives relevant info. Eg:
```
project_name: masks
train_set: train
val_set: val
num_gpus: 1  # 0 means using cpu, 1-N means using gpus 

# mean and std in RGB order, actually this part should remain unchanged as long as your dataset is similar to coco.
mean: [0.485, 0.456, 0.406]
std: [0.229, 0.224, 0.225]

# this is coco anchors, change it if necessary
anchors_scales: '[2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)]'
anchors_ratios: '[(1.0, 1.0), (1.4, 0.7), (0.7, 1.4)]'

# objects from all labels from your dataset with the order from your annotations.
# its index must match your dataset's category_id.
# category_id is one_indexed,
obj_list: ['mask', 'no_mask']
``` 

In [ ]:
!wget -q https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/maskd/v0.1/train_images.zip
!wget -q https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/maskd/v0.1/val_images.zip
!wget -q https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/maskd/v0.1/test_images.zip
!unzip -q train_images.zip
!unzip -q val_images.zip
!unzip -q test_images.zip
!wget -q https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/maskd/v0.1/train.json
!wget -q https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/maskd/v0.1/val.json
!wget -q https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/maskd/v0.1/test.json

In [ ]:
!mkdir datasets/masks
!mv train_images datasets/masks
!mv val_images datasets/masks
!mv test_images datasets/masks
!mkdir datasets/masks/annotations
!mv train.json datasets/masks/annotations
!mv val.json datasets/masks/annotations
!mv test.json datasets/masks/annotations

In [ ]:
!mv datasets/masks/annotations/train.json datasets/masks/annotations/instances_train.json
!mv datasets/masks/annotations/val.json datasets/masks/annotations/instances_val.json
!mv datasets/masks/annotations/test.json datasets/masks/annotations/instances_test.json


### Download Pretrained Weights

In [ ]:
! mkdir weights
!wget https://github.com/zylo117/Yet-Another-EfficientDet-Pytorch/releases/download/1.0/efficientdet-d6.pth -O weights/efficientdet-d6.pth

## Training

In [ ]:
# consider this is a simple dataset, train head will be enough.
! python train.py -c 6 -p masks  --lr 1e-3 --batch_size 2 --load_weights weights/efficientdet-d6.pth  --num_epochs 20 --head_only True

# the loss will be high at first
# don't panic, be patient,
# just wait for a little bit longer

## Evaluation

In [ ]:
! python coco_eval.py -c 6 -p masks -w logs/masks/efficientdet-d6_19_6500.pth

running coco-style evaluation on project masks, weights logs/masks/efficientdet-d6_19_6500.pth...
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
100% 120/120 [00:37<00:00,  3.20it/s]
Loading and preparing results...
DONE (t=0.05s)
creating index...
index created!
BBox
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.32s).
Accumulating evaluation results...
DONE (t=0.05s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.351
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.751
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.237
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.472
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.374
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.347
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.243
 Ave

## Testing

In [ ]:
def evaluate_coco(img_path, set_name, image_ids, coco, model, threshold=0.05):
    results = []

    regressBoxes = BBoxTransform()
    clipBoxes = ClipBoxes()

    for image_id in tqdm(image_ids):
        image_info = coco.loadImgs(image_id)[0]
        image_path = img_path + image_info['file_name']

        ori_imgs, framed_imgs, framed_metas = preprocess(image_path, max_size=input_sizes[compound_coef])
        x = torch.from_numpy(framed_imgs[0])

        if use_cuda:
            x = x.cuda(gpu)
            if use_float16:
                x = x.half()
            else:
                x = x.float()
        else:
            x = x.float()

        x = x.unsqueeze(0).permute(0, 3, 1, 2)
        features, regression, classification, anchors = model(x)

        preds = postprocess(x,
                            anchors, regression, classification,
                            regressBoxes, clipBoxes,
                            threshold, nms_threshold)
        
        if not preds:
            continue

        preds = invert_affine(framed_metas, preds)[0]

        scores = preds['scores']
        class_ids = preds['class_ids']
        rois = preds['rois']

        if rois.shape[0] > 0:
            # x1,y1,x2,y2 -> x1,y1,w,h
            rois[:, 2] -= rois[:, 0]
            rois[:, 3] -= rois[:, 1]

            bbox_score = scores

            for roi_id in range(rois.shape[0]):
                score = float(bbox_score[roi_id])
                label = int(class_ids[roi_id])
                box = rois[roi_id, :]

                image_result = {
                    'image_id': image_id,
                    'category_id': label + 1,
                    'score': float(score),
                    'bbox': box.tolist(),
                }

                results.append(image_result)

    if not len(results):
        raise Exception('the model does not provide any valid output, check model architecture and the data input')

    # write output
    filepath = f'test_bbox_results.json'
    if os.path.exists(filepath):
        os.remove(filepath)
    json.dump(results, open(filepath, 'w'), indent=4)

In [ ]:
import json
import os

import argparse
import torch
import yaml
from tqdm import tqdm
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

from backbone import EfficientDetBackbone
from efficientdet.utils import BBoxTransform, ClipBoxes
from utils.utils import preprocess, invert_affine, postprocess, boolean_string


In [ ]:
project_name = 'masks'

params = yaml.safe_load(open(f'projects/{project_name}.yml'))
obj_list = params['obj_list']
nms_threshold = 0.5
weights_path = 'logs/masks/efficientdet-d4_19_6500.pth'
compound_coef = 4
use_float16 = False
SET_NAME = params['val_set']
use_cuda = True
override_prev_results = True
VAL_GT = f'datasets/{params["project_name"]}/annotations/instances_test.json'
VAL_IMGS = f'datasets/{params["project_name"]}/test_images/'
MAX_IMAGES = 10000
coco_gt = COCO(VAL_GT)
gpu = 0
image_ids = coco_gt.getImgIds()[:MAX_IMAGES]
input_sizes = [512, 640, 768, 896, 1024, 1280, 1280, 1536, 1536]

if override_prev_results or not os.path.exists(f'test_bbox_results.json'):
    model = EfficientDetBackbone(compound_coef=compound_coef, num_classes=len(obj_list),
                                  ratios=eval(params['anchors_ratios']), scales=eval(params['anchors_scales']))
    model.load_state_dict(torch.load(weights_path, map_location=torch.device('cpu')))
    model.requires_grad_(False)
    model.eval()

    if use_cuda:
        model.cuda(gpu)

        if use_float16:
            model.half()

    evaluate_coco(VAL_IMGS, SET_NAME, image_ids, coco_gt, model)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


100%|██████████| 774/774 [01:41<00:00,  7.66it/s]
